## Converting TF model to TF Lite model

### Import Packages

In [1]:
from keras.layers import TFSMLayer
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.xception import preprocess_input
import numpy as np
import tensorflow as tf
import tensorflow.lite as tflite

2025-07-18 09:00:21.562578: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-18 09:00:21.566033: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-18 09:00:21.574952: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752829221.589426    9927 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752829221.593657    9927 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752829221.605937    9927 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

### Test the trained model for inference

In [3]:
img = load_img('test/hat/2a12baab-f020-42e3-8e6b-5d82e3ed0b55.jpg',target_size = (299,299))
x= np.array(img)
X = np.array([x])
X = preprocess_input(X)
model = TFSMLayer("xception_v1_34_0.883", call_endpoint="serving_default")
output_dict = model(X)
output = output_dict['dense_7'].numpy().squeeze()
classes = ['dress', 'hat', 'longsleeve', 'outwear', 'pants', 'shirt', 'shoes', 'shorts', 'skirt', 't-shirt']
pred_index = np.argmax(output)
predicted_class = classes[pred_index].capitalize()
predicted_score = output[pred_index]
print(f"Predicted class: {predicted_class} (score: {predicted_score:.4f})")

Predicted class: Hat (score: 4.8221)


### Converting model to TFLite

In [4]:
converter = tf.lite.TFLiteConverter.from_saved_model("xception_v1_34_0.883")
tflite_model = converter.convert()
with open("clothing_model.tflite", "wb") as f:
    f.write(tflite_model)

W0000 00:00:1752829268.819278    9927 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1752829268.819319    9927 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-07-18 09:01:08.820171: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: xception_v1_34_0.883
2025-07-18 09:01:08.835572: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-07-18 09:01:08.835587: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: xception_v1_34_0.883
I0000 00:00:1752829268.915065    9927 mlir_graph_optimization_pass.cc:425] MLIR V1 optimization pass is not enabled
2025-07-18 09:01:08.926908: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-07-18 09:01:09.263435: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: xception_v1_34_0.883
2025-07-18 09:01:09.357329: I tensorflow/cc/saved_model/loader.cc:471]

#### Testing the TFLite model

In [5]:
interpreter = tflite.Interpreter(model_path = 'clothing_model.tflite')
interpreter.allocate_tensors()

/usr/local/python/3.12.1/lib/python3.12/site-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [6]:
input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']
interpreter.set_tensor(input_index,X)
interpreter.invoke()
output = interpreter.get_tensor(output_index)
classes = ['dress', 'hat', 'longsleeve', 'outwear', 'pants', 'shirt', 'shoes', 'shorts', 'skirt', 't-shirt']
pred_index = np.argmax(output[0])
predicted_class = classes[pred_index].capitalize()
predicted_score = output[0][pred_index]
print(f"Predicted class: {predicted_class} (score: {predicted_score:.4f})")

Predicted class: Hat (score: 4.8221)


### Removing TF dependency completely

##### Restart the kernel to remove all pre-imported packages

In [1]:
!pip install ai-edge-litert

In [2]:
from PIL import Image
import numpy as np
from ai_edge_litert.interpreter import Interpreter

In [3]:
def preprocess_input(x):
    x /= 127.5
    x -= 1.
    return x

In [4]:
hat_url = 'test/hat/2a12baab-f020-42e3-8e6b-5d82e3ed0b55.jpg'
with Image.open(hat_url) as img:
    img = img.resize((299,299),Image.NEAREST)
x= np.array(img, dtype = 'float32')
X = np.array([x])
X= preprocess_input(X)

In [5]:
interpreter = Interpreter(model_path = 'clothing_model.tflite')
interpreter.allocate_tensors()
input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']
interpreter.set_tensor(input_index,X)
interpreter.invoke()
output = interpreter.get_tensor(output_index)
classes = ['dress', 'hat', 'longsleeve', 'outwear', 'pants', 'shirt', 'shoes', 'shorts', 'skirt', 't-shirt']
pred_index = np.argmax(output[0])
predicted_class = classes[pred_index].capitalize()
predicted_score = output[0][pred_index]
print(f"Predicted class: {predicted_class} (score: {predicted_score:.4f})")

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Predicted class: Hat (score: 4.8221)
